# Binance Long Only Strategy

A not profitable demo strategy using the binance python wrapper and telegram notifications

In [1]:
# Demo Account API and Secret Key
api_key = 'api key'
api_secret = 'secret key'

In [2]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import requests

In [3]:
client = Client(api_key = api_key, api_secret = api_secret, tld = "com", testnet = True)


# The  LongOnlyTrader Class

In [4]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 
        self.trade_values = [] 
        
        #*****************strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
         
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
        
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** strategy definition ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    
    def execute_trades(self): 
        
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")
                
                
                # Telegram message
                time = pd.to_datetime(order["transactTime"], unit = "ms")  
                base_units = float(order["executedQty"])
                quote_units = float(order["cummulativeQuoteQty"])
                price = round(quote_units / base_units, 5)
                message= 'Going Long'
                base_url = 'https://api.telegram.org/bot5417801030:AAHUvpJLh9rlzobVq6OOaIB7hMlMeguMBcc/sendMessage?chat_id=-1001577030962&text={},{},{}'.format(message,time,price)
                requests.get(base_url)

            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
                
                # Telegram message
                base_units = float(order["executedQty"])
                quote_units = float(order["cummulativeQuoteQty"])
                time = pd.to_datetime(order["transactTime"], unit = "ms")         
                price = round(quote_units / base_units, 5)
                message='Going Neutral'
                base_url = 'https://api.telegram.org/bot5417801030:AAHUvpJLh9rlzobVq6OOaIB7hMlMeguMBcc/sendMessage?chat_id=-1001577030962&text={},{},{}'.format(message,time,price)
                requests.get(base_url)
            self.position = 0
    
    def report_trade(self, order, going):
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, cum_profits))
        print(100 * "-" + "\n")

In [5]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1656615960723,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.03000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9428.01778008', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [6]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]
units = 0.01
position = 0

In [7]:
trader = LongOnlyTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                        volume_thresh = volume_thresh, units = units, position = position)
trader

In [8]:
trader.start_trading(historical_days = 1/24)

...................

----------------------------------------------------------------------------------------------------
2022-06-30 20:52:00.593000 | GOING LONG
2022-06-30 20:52:00.593000 | Base_Units = 0.01 | Quote_Units = 186.9009 | Price = 18690.09 
2022-06-30 20:52:00.593000 | Profit = 0 | CumProfits = 0.0 
----------------------------------------------------------------------------------------------------

.......

In [10]:
trader.twm.stop()

In [14]:
trader.prepared_data.tail(10)

,Close,Volume,returns,vol_ch,position
Date,,,,,
2022-06-30 17:35:00,19174.10,44.49322,0.001383,0.168652,0
2022-06-30 17:36:00,19187.79,40.69518,0.000714,-0.089227,0
2022-06-30 17:37:00,19194.54,76.30077,0.000352,0.628573,0
2022-06-30 17:38:00,19182.92,23.52025,-0.000606,-1.176821,1
2022-06-30 17:39:00,19196.91,34.95943,0.000729,0.396326,0
2022-06-30 17:40:00,19211.60,59.24346,0.000765,0.527467,0
2022-06-30 17:41:00,19185.10,46.93174,-0.001380,-0.232961,1
2022-06-30 17:42:00,19157.58,76.74436,-0.001435,0.491786,1
2022-06-30 17:43:00,19144.05,44.68804,-0.000706,-0.540774,1


In [15]:
trader.trade_values

[-190.9462, 190.9769, -191.4762, 191.7042, -191.8292, 191.969, -191.851]

In [16]:
np.sum(trader.trade_values)

-191.45250000000001

In [17]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1656610920434,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.01000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9808.66110668', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [1]:
pip show python-binance

Name: python-binance
Version: 1.0.15
Summary: Binance REST API python implementation
Home-page: https://github.com/sammchardy/python-binance
Author: Sam McHardy
Author-email: 
License: MIT
Location: c:\users\matt\anaconda3\lib\site-packages
Requires: aiohttp, dateparser, requests, six, ujson, websockets
Required-by: 
Note: you may need to restart the kernel to use updated packages.
